<a href="https://colab.research.google.com/github/gabrielanatalia/PortfolioVisualizer/blob/main/Portfolio_Visualizer_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Portfolio Visualizer**



Click the triangle icon to run each code block, in sequential order.

# Setup

In [1]:
# @title Load libraries and functions
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from datetime import date, timedelta
import statsmodels.api as sm
from scipy.stats import norm
from scipy.optimize import minimize
from scipy.optimize import fsolve
import itertools
import plotly.graph_objects as go
import yfinance as yf
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap
from matplotlib import colors
import statsmodels.formula.api as smf
from dateutil.parser import parse
import pytz
import re
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display, clear_output
import time

!git clone https://github.com/gabrielanatalia/Projects/
%cd /content/Projects
import sys
sys.path.append('/content/Projects')
import port_cons as pc

import warnings
warnings.filterwarnings("ignore")

def convert_to_datetime(input_str, parserinfo=None):
    return parse(input_str, parserinfo=parserinfo)

TOLERANCE = 1e-10

def _allocation_risk(weights, covariances):

    portfolio_risk = np.sqrt((weights * covariances * weights.T))[0, 0]

    return portfolio_risk

def _assets_risk_contribution_to_allocation_risk(weights, covariances):

    portfolio_risk = _allocation_risk(weights, covariances)

    assets_risk_contribution = np.multiply(weights.T, covariances * weights.T) \
        / portfolio_risk

    return assets_risk_contribution

def _risk_budget_objective_error(weights, args):
    covariances = args[0]
    assets_risk_budget = args[1]
    weights = np.matrix(weights)

    portfolio_risk = _allocation_risk(weights, covariances)

    assets_risk_contribution = \
        _assets_risk_contribution_to_allocation_risk(weights, covariances)

    assets_risk_target = \
        np.asmatrix(np.multiply(portfolio_risk, assets_risk_budget))

    error = sum(np.absolute(assets_risk_contribution - assets_risk_target.T))[0, 0]
    return error

def _get_risk_parity_weights(covariances, assets_risk_budget, initial_weights):

    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0},{'type': 'ineq', 'fun': lambda x: x})

    optimize_result = minimize(fun=_risk_budget_objective_error,
                               x0=initial_weights,
                               args=[covariances, assets_risk_budget],
                               method='SLSQP',
                               constraints=constraints,
                               tol=TOLERANCE,
                               options={'disp': False})

    weights = optimize_result.x
    print(optimize_result.message)
    return weights

def rebal_wgt_riskparity(returns_data, start_date, end_date, rebal_months=[4,10], halflife=3.5, annualized=252, shrink_covar=False):
    num_of_assets = len(returns_data.columns)

    ret_data_filtered = returns_data.loc[start_date:end_date]
    first_date = returns_data.index[0]
    tickers = list(returns_data.columns)

    weights = pd.DataFrame(0, index=ret_data_filtered.index, columns=ret_data_filtered.columns)
    weights.index = pd.to_datetime(weights.index, format='%Y-%m-%d')
    reb_flag = pd.DataFrame(0, index=weights.index, columns=['reb_flag'])

    alpha = 1 - math.exp(math.log(0.5) / (halflife * annualized))
    span = (2 / alpha) - 1

    for i in range(len(ret_data_filtered)):
        curr_date = ret_data_filtered.index[i]
        curr_date_format = curr_date.strftime('%Y-%m-%d')

        if (weights.index[i].month in rebal_months and weights.index[i-1].month != weights.index[i].month) or i==0:

            if shrink_covar:
                shrink_covar_matrix_model = LedoitWolf().fit(returns_data.loc[first_date:curr_date])
                covar_ann = pd.DataFrame(shrink_covar_matrix_model.covariance_ * annualized, index=tickers, columns=tickers)
                covar_ann.columns = tickers
                covar_ann.index = tickers
            else:
                exp_cov_matrix = returns_data.loc[first_date:curr_date].ewm(span=span).cov(pairwise=True).iloc[-num_of_assets:]
                covar_ann = exp_cov_matrix * annualized
                covar_ann.columns = tickers
                covar_ann.index = tickers

            valid_covar_ann = covar_ann.dropna(how='all')
            valid_covar_ann = valid_covar_ann.dropna(axis=1, how='all')
            valid_num_assets = len(valid_covar_ann)
            valid_tickers = valid_covar_ann.columns

            assets_risk_budget = np.ones([valid_num_assets]) / valid_num_assets
            initial_weights = np.ones([valid_num_assets]) / valid_num_assets

            # print(valid_covar_ann)
            new_wgt = _get_risk_parity_weights(valid_covar_ann.values, assets_risk_budget, initial_weights)
            new_wgt = pd.DataFrame(new_wgt).T
            new_wgt.columns = valid_tickers

            for ticker in valid_tickers:
                weights.at[curr_date_format, ticker] = new_wgt[ticker].values

            reb_flag.loc[curr_date_format] = True

        else:
            weights.iloc[i] = weights.iloc[i-1] * (1+ ret_data_filtered.iloc[i].fillna(0))
            weights_sum = weights.iloc[i].sum()
            weights.iloc[i] /= weights_sum
            reb_flag.iloc[i] = False

    weights = weights.rename(columns={c: c + '_wgt' for c in weights.columns})
    weights = pd.concat([reb_flag, weights], axis=1)

    return weights

def combine_backtest_data(portfolio_names):
    # combined portfolio returns
    all_port_ret = {}

    for portfolio in portfolio_names:
        perf_df = globals()[f"{portfolio}_perf"]
        ret_col = [col for col in perf_df.columns if col.endswith('_port_ret')]
        all_port_ret[portfolio] = perf_df[ret_col]

    df_all_port_ret = pd.concat(all_port_ret.values(), keys=all_port_ret.keys(), axis=1)
    df_all_port_ret.columns = [col[0] for col in df_all_port_ret.columns]

    # combined portfolio weights
    all_port_weights = {}

    for portfolio in portfolio_names:
        wgt_df = globals()[f"{portfolio}_wgt"]
        # wgt_col = [col for col in wgt_df.columns if col.endswith('_wgt')]
        # all_port_weights[portfolio] = wgt_df[wgt_col]
        all_port_weights[portfolio] = wgt_df

    df_all_port_weights = pd.concat(all_port_weights.values(), keys=all_port_weights.keys(), axis=1)

    # combined portfolio backtest daata
    all_port_bt = {}

    for portfolio in portfolio_names:
        bt_df = globals()[f"{portfolio}_perf"]
        all_port_bt[portfolio] = bt_df

    df_all_port_bt = pd.concat(all_port_bt.values(), keys=all_port_bt.keys(), axis=1)
    return df_all_port_ret, df_all_port_weights, df_all_port_bt




########################################################################################################################

# read asseet class, sector, country data from csv
data_path = '/content/Projects/Data/'
df_sec_list = pd.read_csv(data_path + 'PV_sec_list.csv')
df_asset_class = pd.read_csv(data_path + 'PV_asset_class.csv', index_col=0)
df_sector = pd.read_csv(data_path + 'PV_sector.csv', index_col=0)
df_country = pd.read_csv(data_path + 'PV_country.csv', index_col=0)

Cloning into 'Projects'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 79 (delta 39), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (79/79), 10.50 MiB | 4.94 MiB/s, done.
Resolving deltas: 100% (39/39), done.
/content/Projects


In [3]:
# @title See full list of securities

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

out = widgets.Output()

# Define a function to display securities for a given category
def display_securities_df(category):
    with out:
        out.clear_output(wait=True)  # Clear previous output
        filtered_df = df_sec_list[df_sec_list['Category'] == category][['Ticker', 'Name']].sort_values(by='Ticker', ascending=True)
        display(filtered_df.reset_index(drop=True))

# Create buttons for each category
equity_button = widgets.Button(description="Equities")
fixed_income_button = widgets.Button(description="Fixed income")
commodities_button = widgets.Button(description="Commodities")
multi_asset_button = widgets.Button(description="Multi-asset")
index_button = widgets.Button(description="Index")

# Assign the display_securities_df function to each button's on_click event
equity_button.on_click(lambda b: display_securities_df("Equities"))
fixed_income_button.on_click(lambda b: display_securities_df("Fixed income"))
commodities_button.on_click(lambda b: display_securities_df("Commodities"))
multi_asset_button.on_click(lambda b: display_securities_df("Multi-asset"))
index_button.on_click(lambda b: display_securities_df("Index"))

# Display the buttons and output widget
display(widgets.HBox([equity_button, fixed_income_button, commodities_button, multi_asset_button,index_button]))
display(out)

Output()

In [4]:
# @title Check if a security is in the list

etf_ticker_input = widgets.Text(
    value='',
    placeholder='Enter ETF ticker',
    description='ETF Ticker:',
    disabled=False
)

# Create an output widget to display the result
output_check = widgets.Output()

# Define a function to check if the ETF ticker is in df_sec_list
def check_security(b):
  with output_check:
    clear_output(wait=True)
    ticker = etf_ticker_input.value.upper()  # Convert to uppercase for case-insensitivity
    if ticker in df_sec_list['Ticker'].values:
      print(f"{ticker} is in the list.")
    else:
      print(f"{ticker} is not in the list.")

# Create a button to trigger the check
check_button = widgets.Button(description="Check Security")
check_button.on_click(check_security)

# Display the widgets
display(etf_ticker_input, check_button, output_check)

Text(value='', description='ETF Ticker:', placeholder='Enter ETF ticker')

Button(description='Check Security', style=ButtonStyle())

Output()

# Parameters





In [5]:
# @title Choose data source
# @markdown Note:
# @markdown - Bloomberg: Provides total return data, but not updated in real-time.
# @markdown - Yahoo finance: Provides live data (as of last close), but includes only price returns.

data_source_widget = widgets.RadioButtons(
       options=['Yahoo Finance', 'Bloomberg'],
       description='Data Source:',
       disabled=False)
display(data_source_widget)


RadioButtons(description='Data Source:', options=('Yahoo Finance', 'Bloomberg'), value='Yahoo Finance')

Input the data parameters

*   Start date: format - DD/MM/YYYY
*   End date: format - DD/MM/YYYY
*   Underlying tickers, format - ETF1, ETF2, ETF3, ... | e.g. SPY, EFA, QQQ,
RSP, IEMG
*   Benchmark ticker format - ETF1, ETF2, ETF3, ... | e.g. ACWI, URTH, SPY
*   Observation window (days): the number of days after the specified start date used to initialize the optimization process - this ensures enough data is available before the backtest begins (i.e. if lookback window is 365 days and start date is Jan 1 2010, the actual backtest will begin in Jan 1 2011)


❗ If no start and end date is specified, the backtest will begin from the earliest common inception until latest available data

In [6]:
# @title Input data parameters

# Widget for start date
start_date_widget = widgets.DatePicker(
    description='Start Date:',
    # value=parse("2010-01-01").date(),  # Default value
    layout=widgets.Layout(width='300px'),  # Adjust width as needed
    style={'description_width': 'initial'}  # Allow description to take full width
)

# Widget for end date
end_date_widget = widgets.DatePicker(
    description='End Date:',
    # value=parse("2024-10-31").date(),  # Default value
    layout=widgets.Layout(width='300px'),  # Adjust width as needed
    style={'description_width': 'initial'}  # Allow description to take full width
)

# Widget for tickers
tickers_widget = widgets.Text(
    value='',
    description='Tickers:',
    disabled=False,
    layout=widgets.Layout(width='300px'),  # Adjust width as needed
    style={'description_width': 'initial'}  # Allow description to take full width
)

# Widget for benchmark tickers
bm_tickers_widget = widgets.Text(
    value='',
    description='Benchmark Tickers:',
    disabled=False,
    layout=widgets.Layout(width='300px'),  # Adjust width as needed
    style={'description_width': 'initial'}  # Allow description to take full width
)

# Widget for backtest offset days
backtest_offset_days_widget = widgets.IntText(
    value=365,
    description='Observation window (Days):',
    disabled=False,
    layout=widgets.Layout(width='300px'),  # Adjust width as needed
    style={'description_width': 'initial'}  # Allow description to take full width
)

# Display the widgets
display(start_date_widget)
display(end_date_widget)
display(tickers_widget)
display(bm_tickers_widget)
display(backtest_offset_days_widget)


# Function to update variables based on widget values
def update_variables(change):
    global start_date, end_date, tickers, bm_tickers, all_tickers, backtest_offset_days

    start_date = start_date_widget.value
    end_date = end_date_widget.value

    tickers = re.split(r'\s*,\s*', tickers_widget.value)
    bm_tickers = re.split(r'\s*,\s*', bm_tickers_widget.value)

    all_tickers = list(set(tickers + bm_tickers))

    backtest_offset_days = backtest_offset_days_widget.value


# Observe widget changes and update variables
start_date_widget.observe(update_variables, names='value')
end_date_widget.observe(update_variables, names='value')
tickers_widget.observe(update_variables, names='value')
bm_tickers_widget.observe(update_variables, names='value')
backtest_offset_days_widget.observe(update_variables, names='value')

# Initial variable update
update_variables(None)

DatePicker(value=None, description='Start Date:', layout=Layout(width='300px'), style=DescriptionStyle(descrip…

DatePicker(value=None, description='End Date:', layout=Layout(width='300px'), style=DescriptionStyle(descripti…

Text(value='', description='Tickers:', layout=Layout(width='300px'), style=DescriptionStyle(description_width=…

Text(value='', description='Benchmark Tickers:', layout=Layout(width='300px'), style=DescriptionStyle(descript…

IntText(value=365, description='Observation window (Days):', layout=Layout(width='300px'), style=DescriptionSt…

In [20]:
# @title Download data
# df_all = yf.download(all_tickers, start=start_date, end=end_date)['Adj Close']
# if start_date is None and end_date is None:
#     df_all = yf.download(all_tickers, period='max')['Adj Close']
# else:
#     df_all = yf.download(all_tickers, start=start_date, end=end_date)['Adj Close']

# df_all.index = df_all.index.tz_localize(None)
# ret_all = df_all.pct_change().dropna()
# ret_bm = ret_all[bm_tickers]

# earliest_start_date = ret_all.index.min()
# start_date_timestamp = pd.Timestamp(start_date)

# # Compare earliest_start_date with start_date_timestamp
# if earliest_start_date > start_date_timestamp:
#     print('\n', 'Earliest start date: ', earliest_start_date)
# else:
#     pass


def download_data(data_source, start_date, end_date):
  global ret_all, ret_bm, earliest_start_date # Declare as global
  if data_source == 'Yahoo Finance':
    if start_date is None and end_date is None:
      df_all = yf.download(all_tickers, period='max')['Adj Close']
    else:
      df_all = yf.download(all_tickers, start=start_date, end=end_date)['Adj Close']

    df_all.index = df_all.index.tz_localize(None)
    ret_all = df_all.pct_change().dropna()
    ret_bm = ret_all[bm_tickers]


  elif data_source == 'Bloomberg':
    ret_all = pd.read_csv(data_path + 'PV_daily_ret.csv',header=0, index_col='Date', parse_dates=['Date'],dayfirst=True)[all_tickers]/100
    if start_date is None and end_date is None:
      ret_all = ret_all = ret_all.dropna()
    else:
      start_date = start_date.strftime('%Y-%m-%d')
      end_date = end_date.strftime('%Y-%m-%d')
      ret_all = ret_all.loc[start_date:end_date].dropna()

    ret_bm = ret_all[bm_tickers]

  earliest_start_date = ret_all.index.min()
  start_date_timestamp = pd.Timestamp(start_date)

  # Compare earliest_start_date with start_date_timestamp
  if earliest_start_date > start_date_timestamp:
    print('\n', 'Earliest start date: ', earliest_start_date)
  else:
    pass

download_button = widgets.Button(description="Download Data")

def on_download_button_clicked(b):
  download_data(data_source_widget.value, start_date, end_date)

download_button.on_click(on_download_button_clicked)
display(download_button)



Button(description='Download Data', style=ButtonStyle())


 Earliest start date:  2020-01-02 00:00:00


# Constituent performance

In [22]:
# @title Generate constituent performance
from google.colab import widgets as gc_widgets

# perf summary
constituent_summary = pc.performance_summary_constituents(ret_all, start_date=start_date, end_date=end_date, frequency='daily')
# calendar year returns
yearly_df = pc.constituents_calendar_year_returns(ret_all, frequency='daily')

def generate_constituents_key_metrics_func(b):
    clear_output(wait=True)  # Clear previous output
    generate_constituents_key_metrics()

def generate_constituents_key_metrics():
    # Define tab names
    tab_names = ["Key Summary", "Calendar Year Returns", "Monthly Returns", "Cumulative Returns", "Return/Risk", "Returns Distribution", "Volatility", "Drawdown", "Correlation"]

    # Create TabBar
    t = gc_widgets.TabBar(tab_names)

    # Populate tabs with content
    with t.output_to(tab_names[0]):  # Key Summary
        display(constituent_summary.T)

    with t.output_to(tab_names[1]):  # Calendar Year Returns
        print('Date range: ', ret_all.index.min(), ' - ', ret_all.index.max(), '\n')
        display(yearly_df.pipe(pc.apply_style_heatmap_ret))

    with t.output_to(tab_names[2]):  # Monthly Returns
      for i in ret_all.columns:
        security_returns = ret_all[[i]]
        monthly_performance = pc.monthly_performance_table(security_returns)
        print(i)
        display(monthly_performance.pipe(pc.apply_style_heatmap_ret))
        print('\n')

    with t.output_to(tab_names[3]):  # Cumulative Returns
        display(pc.plot_cumulative_returns(ret_all, show_data=True))

    with t.output_to(tab_names[4]):  # Return/risk scatter plot
        display(pc.plot_return_risk_scatter_maxrange(ret_all))

    with t.output_to(tab_names[5]):  # Returns Distribution
        pc.plot_returns_distribution_boxplot(ret_all)

    with t.output_to(tab_names[6]):  # Rolling volatility
        pc.plot_rolling_volatility(ret_all, window=261, title='Rolling 1Y Volatility')

    with t.output_to(tab_names[7]):  # Drawdown
        display(pc.plot_drawdowns(ret_all, show_data=True))

    with t.output_to(tab_names[8]):  # Correlation
        pc.plot_correlation_heatmap(ret_all)

    display(t)


constituents_key_metrics_button = widgets.Button(description="Generate Constituents Key Metrics", layout=widgets.Layout(width='auto'))
constituents_key_metrics_button.on_click(generate_constituents_key_metrics_func)
display(constituents_key_metrics_button)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Portfolio configuration & parameters

In [23]:
# @title Input portfolio parameters
# @markdown If 'Allocation' = 'Custom weight' , weight input field will be visible
def create_portfolio_widgets(portfolio_number):
    """Creates and displays widgets for a single portfolio."""
    display(widgets.HTML(f"<h3>Portfolio {portfolio_number}</h3>"))

    tickers_widget = widgets.Text(
        value='',
        description='Tickers:',
        disabled=False
    )
    allocation_type_widget = widgets.Dropdown(
        options=['Max sharpe', 'Min vol', 'Risk parity', 'Equal weight', 'Custom weight'],
        value='Max sharpe',  # Set default value based on portfolio_number or user preference
        description='Allocation:',
        disabled=False,
    )
    name_widget = widgets.Text(
        value=f'P{portfolio_number}',  # Set default name based on portfolio_number or user preference
        description='Name:',
        disabled=False
    )
    min_weight_widget = widgets.FloatText(
        value=0.0,
        description='Min Weight:',
        disabled=False
    )
    max_weight_widget = widgets.FloatText(
        value=1.0,
        description='Max Weight:',
        disabled=False
    )

    custom_weight_widget = widgets.Text(value='',  # Initial value can be empty
        description='Input Weight:',
        disabled=False,
        layout=widgets.Layout(visibility='hidden') # Initially hidden
    )

    def toggle_custom_weight_visibility(change):
        if allocation_type_widget.value == 'Custom weight':
            custom_weight_widget.layout.visibility = 'visible'
        else:
            custom_weight_widget.layout.visibility = 'hidden'

    # Observe the allocation_type_widget for changes
    allocation_type_widget.observe(toggle_custom_weight_visibility, names='value')

    display(tickers_widget)
    display(allocation_type_widget)
    display(name_widget)
    display(min_weight_widget)
    display(max_weight_widget)

    # Display the custom_weight_widget
    display(custom_weight_widget)


    return {
        'tickers': tickers_widget,
        'allocation_type': allocation_type_widget,
        'name': name_widget,
        'min_weight': min_weight_widget,
        'max_weight': max_weight_widget,
        'custom_weight': custom_weight_widget  # Add custom_weight_widget to the dictionary
    }

# Create widgets for 5 portfolios
portfolio_widgets = {}
for i in range(1, 6):
    portfolio_widgets[i] = create_portfolio_widgets(i)


HTML(value='<h3>Portfolio 1</h3>')

Text(value='', description='Tickers:')

Dropdown(description='Allocation:', options=('Max sharpe', 'Min vol', 'Risk parity', 'Equal weight', 'Custom w…

Text(value='P1', description='Name:')

FloatText(value=0.0, description='Min Weight:')

...

Dropdown(description='Allocation:', options=('Max sharpe', 'Min vol', 'Risk parity', 'Equal weight', 'Custom w…

Text(value='P5', description='Name:')

FloatText(value=0.0, description='Min Weight:')

FloatText(value=1.0, description='Max Weight:')

Text(value='', description='Input Weight:', layout=Layout(visibility='hidden'))

In [24]:
# @title Confirm portfolio parameters
def get_portfolio_specs():
    portfolio_specs = {}
    for i in range(1, 6):
        portfolio_specs[i] = {
            'tickers': re.split(r'\s*,\s*', portfolio_widgets[i]['tickers'].value),
            'allocation_type': portfolio_widgets[i]['allocation_type'].value,
            'name': portfolio_widgets[i]['name'].value,
            'min_weight': float(portfolio_widgets[i]['min_weight'].value),
            'max_weight': float(portfolio_widgets[i]['max_weight'].value),
            'custom_weight': portfolio_widgets[i]['custom_weight'].value  # Get the value of the custom weight widget
        }

        if portfolio_specs[i]['allocation_type'] == 'Custom weight':
            try:
                portfolio_specs[i]['custom_weight'] = [float(x) for x in portfolio_specs[i]['custom_weight'].split(',')]
            except ValueError:
                print("Invalid custom weight input. Please enter comma-separated values and ensure values sum to 1 (e.g 0.2,0.3,0.1,0.2,0.2)")
                # You might want to handle the error more gracefully here.

    return portfolio_specs

# Function to execute on button click
def on_confirm_button_clicked(b):
  display(df_portfolio_specs)
  print('\n')
  print("Portfolio specifications confirmed! ✅")
  # You can add any further actions you want to perform here
  # after confirmation, such as starting the optimization.


# Define constraints function
def create_constraints(min_weight, max_weight):
    return ({
        'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1
    }, {
        'type': 'ineq', 'fun': lambda weights: weights - min_weight
    }, {
        'type': 'ineq', 'fun': lambda weights: max_weight - weights
    })


def create_custom_weight_dict(portfolio_specs, portfolio_number):
  # Creates a dictionary with ticker as key and custom weight as value.
  portfolio = portfolio_specs[portfolio_number]
  tickers = portfolio['tickers']
  custom_weights = portfolio['custom_weight']

  if len(tickers) != len(custom_weights):
    raise ValueError("Number of tickers and custom weights must be equal.")

  custom_weight_dict = dict(zip(tickers, custom_weights))
  return custom_weight_dict

def check_custom_weight_sum(portfolio_specs):

  for portfolio_number, portfolio_data in portfolio_specs.items():
      if portfolio_data['allocation_type'] == 'Custom weight':
          custom_weights = portfolio_data['custom_weight']
          if np.isclose(np.sum(custom_weights), 1.0):
              pass
          else:
              print(f"Portfolio {portfolio_number}: Custom weights do not sum to 1. Please amend.")


portfolio_specs = get_portfolio_specs()
df_portfolio_specs = pd.DataFrame(portfolio_specs).T

# Create a confirmation button
confirm_button = widgets.Button(description="Confirm Portfolio Specs", layout=widgets.Layout(width='auto'))

# Attach the function to the button's on_click event
confirm_button.on_click(on_confirm_button_clicked)

# Display the button
display(confirm_button)

# Call the function to check the custom weight sum
check_custom_weight_sum(portfolio_specs)

Button(description='Confirm Portfolio Specs', layout=Layout(width='auto'), style=ButtonStyle())

,tickers,allocation_type,name,min_weight,max_weight,custom_weight
1,"[SPY, GLD, BLOK, QQQ]",Max sharpe,P1,0.0,0.4,
2,"[SPY, GLD, BLOK, QQQ]",Min vol,P2,0.0,0.4,
3,"[SPY, GLD, BLOK, QQQ]",Risk parity,P3,0.0,1.0,
4,"[SPY, GLD, BLOK, QQQ]",Equal weight,P4,0.0,1.0,
5,"[SPY, GLD, BLOK, QQQ]",Custom weight,P5,0.0,1.0,"[0.4, 0.15, 0.15, 0.3]"




Portfolio specifications confirmed! ✅


In [25]:
# @title Run optimisation

# Create a progress bar
progress_bar = widgets.IntProgress(value=0, min=0, max=5, # Total number of portfolios
                                   description='Optimising:', bar_style='',style={'bar_color': 'navy'},orientation='horizontal')

# Display the progress bar
display(progress_bar)

for i in range(1, 6):
    portfolio = portfolio_specs[i]
    portfolio_name = portfolio['name']
    tickers = portfolio['tickers']
    allocation_type = portfolio['allocation_type']
    min_weight = portfolio['min_weight']
    max_weight = portfolio['max_weight']
    custom_weight = portfolio['max_weight']

    # Create constraints for each portfolio based on the min and max weight
    constraints = create_constraints(min_weight, max_weight)

    # Calculate weights based on allocation type:
    if allocation_type == 'Max sharpe':
        globals()[f"{portfolio_name}_wgt"] = pc.max_sharpe_rebal_wgt(
            ret_all[tickers],
            start_date=earliest_start_date + timedelta(days=backtest_offset_days),
            end_date=end_date,
            constraints_dict=constraints,
            shrink_covar=False,
            rebal_months=[1],
            annualized=261,
            halflife=3.5
        )
    elif allocation_type == 'Min vol':
        globals()[f"{portfolio_name}_wgt"] = pc.min_vol_rebal_wgt(
            ret_all[tickers],
            start_date=earliest_start_date + timedelta(days=backtest_offset_days),
            end_date=end_date,
            constraints_dict=constraints,
            shrink_covar=False,
            rebal_months=[1],
            annualized=261,
            halflife=3.5
        )
    elif allocation_type == 'Risk parity':
        globals()[f"{portfolio_name}_wgt"] = rebal_wgt_riskparity(
            ret_all[tickers],
            start_date=earliest_start_date + timedelta(days=backtest_offset_days),
            end_date=end_date,
            rebal_months=[1],
            halflife=3.5,
            annualized=261,
            shrink_covar=False
        )

    elif allocation_type == 'Equal weight':
        globals()[f"{portfolio_name}_wgt"] = pc.equal_weight_rebal_wgt(
            ret_all[tickers],
            start_date=earliest_start_date + timedelta(days=backtest_offset_days),
            end_date=end_date,
            rebal_months=[1],
        )

    elif allocation_type == 'Custom weight':
        globals()[f"{portfolio_name}_wgt"] = pc.fixed_weight_rebal_wgt(
          ret_all[tickers],
          start_date=earliest_start_date + timedelta(days=backtest_offset_days),
          end_date=end_date,
          fixed_weights=create_custom_weight_dict(portfolio_specs, i),
          rebal_months=[1],
    )

    # Calculate performance:
    globals()[f"{portfolio_name}_perf"] = pc.calc_port_perf(ret_all.loc[earliest_start_date + timedelta(days=backtest_offset_days):end_date][tickers],
                                                            globals()[f"{portfolio_name}_wgt"], name_prefix=portfolio_name)

    # Update the progress bar
    progress_bar.value += 1

clear_output(wait=True) # Clear the output after completion
print("Optimisation complete!")


Optimisation complete!


In [26]:
# @title Historical weights at each point of rebalance
def show_historical_weights(b):
       clear_output(wait=True) # Clear previous output
       for i in range(1, 6):
           portfolio = portfolio_specs[i]
           portfolio_name = portfolio['name']
           print(portfolio_name)
           pc.display_rebal_wgt(globals()[f"{portfolio_name}_wgt"])
           print('\n')

hist_weights_button = widgets.Button(description="Show Historical Weights", layout=widgets.Layout(width='auto'))
hist_weights_button.on_click(show_historical_weights)
display(hist_weights_button)


P1


,reb_flag,SPY_wgt,GLD_wgt,BLOK_wgt,QQQ_wgt
Date,,,,,
2021-01-04 00:00:00,100.00%,0.00%,40.00%,40.00%,20.00%
2022-01-03 00:00:00,100.00%,0.00%,34.72%,25.28%,40.00%
2023-01-03 00:00:00,100.00%,40.00%,40.00%,0.00%,20.00%
2024-01-02 00:00:00,100.00%,15.05%,40.00%,4.95%,40.00%




P2


,reb_flag,SPY_wgt,GLD_wgt,BLOK_wgt,QQQ_wgt
Date,,,,,
2021-01-04 00:00:00,100.00%,40.00%,40.00%,20.00%,0.00%
2022-01-03 00:00:00,100.00%,40.00%,40.00%,0.00%,20.00%
2023-01-03 00:00:00,100.00%,40.00%,40.00%,0.00%,20.00%
2024-01-02 00:00:00,100.00%,40.00%,40.00%,0.00%,20.00%




P3


,reb_flag,SPY_wgt,GLD_wgt,BLOK_wgt,QQQ_wgt
Date,,,,,
2021-01-04 00:00:00,100.00%,19.07%,45.75%,17.92%,17.26%
2022-01-03 00:00:00,100.00%,22.24%,45.78%,13.10%,18.87%
2023-01-03 00:00:00,100.00%,21.90%,48.26%,12.21%,17.62%
2024-01-02 00:00:00,100.00%,22.90%,47.01%,11.89%,18.21%




P4


,reb_flag,SPY_wgt,GLD_wgt,BLOK_wgt,QQQ_wgt
Date,,,,,
2021-01-04 00:00:00,100.00%,25.00%,25.00%,25.00%,25.00%
2022-01-03 00:00:00,100.00%,25.00%,25.00%,25.00%,25.00%
2023-01-03 00:00:00,100.00%,25.00%,25.00%,25.00%,25.00%
2024-01-02 00:00:00,100.00%,25.00%,25.00%,25.00%,25.00%




P5


,reb_flag,SPY_wgt,GLD_wgt,BLOK_wgt,QQQ_wgt
Date,,,,,
2021-01-04 00:00:00,100.00%,40.00%,15.00%,15.00%,30.00%
2022-01-03 00:00:00,100.00%,40.00%,15.00%,15.00%,30.00%
2023-01-03 00:00:00,100.00%,40.00%,15.00%,15.00%,30.00%
2024-01-02 00:00:00,100.00%,40.00%,15.00%,15.00%,30.00%


# Key Data

In [27]:
# @title Generate key metrics

########################################################################################################################

port_names = []  # Initialize an empty list to store names

# Iterate through the portfolio specifications
for portfolio_number, portfolio_data in portfolio_specs.items():
    # Extract the 'name' attribute and append it to the list
    port_names.append(portfolio_data['name'])

all_port_ret, all_port_wgt, all_port_bt = combine_backtest_data(port_names)
ret_bm_filtered = ret_bm.loc[earliest_start_date + timedelta(days=backtest_offset_days):]
ret_bm_filtered.iloc[0] = 0
all_port_ret = pd.concat([all_port_ret, ret_bm], axis=1).dropna()

########################################################################################################################

# perf summary
perf_summary = pc.performance_summary(all_port_ret[port_names], all_port_wgt, all_port_ret[bm_tickers[0]], benchmark_returns=ret_bm,
                                    start_date=earliest_start_date + timedelta(days=backtest_offset_days), end_date=end_date, frequency='daily', rebal_per_year=1)

# calendar year returns
port_yearly_ret = pc.constituents_calendar_year_returns(all_port_ret, frequency='daily')
relative_perf = pc.calculate_relative_per(all_port_ret[port_names], all_port_ret[bm_tickers[0]])
# latest and average allocation
df_ports_alloc_latest = pc.compare_port_alloc(all_port_wgt, port_names, latest_data=True)
df_ports_alloc_avg = pc.compare_port_alloc(all_port_wgt, port_names, latest_data=False)

# portfolio turnover
port_turnover_df = pd.DataFrame()

for portfolio in port_names:
    turnover = pc.portfolio_turnover(all_port_wgt[portfolio])
    port_turnover_df = pd.concat([port_turnover_df, turnover],axis=1)

port_turnover_df.columns = port_names


def generate_key_metrics_func(b):
    clear_output(wait=True)  # Clear previous output
    generate_key_metrics()

def generate_key_metrics():
    # Define tab names
    tab_names = ["Key Summary", "Calendar Year Returns", "Monthly Returns", "Cumulative Returns", "Return/Risk", "Returns Distribution", "Volatility", "Portfolio Drawdown", "Correlation"]

    # Create TabBar
    t = gc_widgets.TabBar(tab_names)

    # Populate tabs with content
    with t.output_to(tab_names[0]):  # Key Summary
        display(perf_summary)

    with t.output_to(tab_names[1]):  # Calendar Year Returns
        print('Calendar year returns')
        display(port_yearly_ret.pipe(pc.apply_style_heatmap_ret))
        print('\n', 'Relative calendar year returns')
        display(pc.apply_style_heatmap_ret(relative_perf, subset=relative_perf.columns[1:]))

    with t.output_to(tab_names[2]):  # Monthly Returns
      for port in port_names:
        portfolio_returns = all_port_ret[[port]]
        monthly_performance = pc.monthly_performance_table(portfolio_returns)
        print(port)
        display(monthly_performance.pipe(pc.apply_style_heatmap_ret))
        print('\n')

    with t.output_to(tab_names[3]):  # Cumulative Returns
        display(pc.plot_cumulative_returns(all_port_ret, show_data=True))

    with t.output_to(tab_names[4]):  # Return/risk scatter plot
        display(pc.plot_return_risk_scatter_maxrange(all_port_ret))

    with t.output_to(tab_names[5]):  # Returns Distribution
        pc.plot_returns_distribution_boxplot(all_port_ret)

    with t.output_to(tab_names[6]):  # Rolling volatility
        pc.plot_rolling_volatility(all_port_ret, window=261, title='Rolling 1Y Volatility')

    with t.output_to(tab_names[7]):  # Portfolio Drawdown
        display(pc.plot_drawdowns(all_port_ret, show_data=True))

    with t.output_to(tab_names[8]):  # Correlation
        pc.plot_correlation_heatmap(all_port_ret)
        pc.plot_rolling_correlation(all_port_ret[port_names], benchmark=all_port_ret[bm_tickers[0]], window=261, title='Rolling 1Y Correlation')

    display(t)


key_metrics_button = widgets.Button(description="Generate Key Metrics", layout=widgets.Layout(width='auto'))
key_metrics_button.on_click(generate_key_metrics_func)
display(key_metrics_button)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [28]:
# @title Generate key holdings & allocations

# asset class exposure
assetclass_analysis_list = [pc.exposure_analysis(all_port_wgt[portfolio], df_asset_class, latest_data=True).rename(columns={0: portfolio}) for portfolio in port_names]
assetclass_breakdown_all = pd.concat(assetclass_analysis_list, axis=1)
assetclass_breakdown_all.columns = port_names
assetclass_breakdown_all = (assetclass_breakdown_all.loc[(assetclass_breakdown_all != 0).any(axis=1)].sort_values(by=assetclass_breakdown_all.columns[0], ascending=False))
# country exposure
country_analysis_list = [pc.exposure_analysis(all_port_wgt[portfolio], df_country, latest_data=True).rename(columns={0: portfolio}) for portfolio in port_names]
country_breakdown_all = pd.concat(country_analysis_list, axis=1)
country_breakdown_all.columns = port_names
country_breakdown_all = (country_breakdown_all.loc[(country_breakdown_all != 0).any(axis=1)].sort_values(by=country_breakdown_all.columns[0], ascending=False))
country_breakdown_all.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage)
# sector exposure
sector_analysis_list = [pc.exposure_analysis(all_port_wgt[portfolio], df_sector, latest_data=True).rename(columns={0: portfolio}) for portfolio in port_names]
sector_breakdown_all = pd.concat(sector_analysis_list, axis=1)
sector_breakdown_all.columns = port_names
sector_breakdown_all = (sector_breakdown_all.loc[(sector_breakdown_all != 0).any(axis=1)].sort_values(by=sector_breakdown_all.columns[0], ascending=False))
sector_breakdown_all.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage)

def generate_key_holdings_func(b):
    clear_output(wait=True)  # Clear previous output
    generate_key_holdings()

def generate_key_holdings():
    # Define tab names
    tab_names = ["Latest Allocation", "Average Allocation", "Historical Allocation", "Asset class exposure", "Country exposure", "Sector exposure"]

    # Create TabBar
    t = gc_widgets.TabBar(tab_names)

    # Populate tabs with content
    with t.output_to(tab_names[0]):  # Latest Allocation
        display(df_ports_alloc_latest.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage))

    with t.output_to(tab_names[1]):  # Average Allocation
        display(df_ports_alloc_avg.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage))

    with t.output_to(tab_names[2]):  # Portfolio Allocation
      for port in port_names:
        print(port)
        pc.plot_weights_and_turnover(all_port_wgt[port], show_data=True, show_turnover=False, show_rebal=True)

    with t.output_to(tab_names[3]):  # Asset class exposure
        display(assetclass_breakdown_all.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage))

    with t.output_to(tab_names[4]):  # Country exposure
        display(country_breakdown_all.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage))

    with t.output_to(tab_names[5]):  # Sector xposure
        display(sector_breakdown_all.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage))

    display(t)

key_holdings_button = widgets.Button(description="Generate Key Holdings & Allocations", layout=widgets.Layout(width='auto'))
key_holdings_button.on_click(generate_key_holdings_func)
display(key_holdings_button)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,P1,P2,P3,P4,P5
Sector,,,,,
Commodities,42.04%,42.23%,48.64%,26.00%,15.93%
Information Technology,26.20%,21.86%,21.14%,31.40%,34.20%
Communication Services,7.43%,6.43%,5.14%,6.96%,8.68%
Consumer Discretionary,6.84%,6.58%,5.38%,7.41%,8.99%
Financials,4.39%,5.46%,8.28%,14.38%,12.25%
Health Care,3.98%,5.28%,3.37%,4.02%,5.94%
Consumer Staples,3.24%,3.35%,2.43%,3.11%,4.17%
Industrials,3.07%,4.20%,2.81%,3.48%,4.88%
Materials,0.88%,1.09%,0.71%,0.85%,1.24%


<IPython.core.display.Javascript object>

# Export data to excel

In [29]:
# @title Input file name and export key data to excel

file_path_widget = widgets.Text(
    value='Portfolio_Visualizer_Output.xlsx',
    description='File Name:',
    disabled=False
)

display(file_path_widget) # Display the widget

# Function to save the data to Excel
def save_to_excel(file_path):
    with pd.ExcelWriter(file_path) as writer:
      constituent_summary.to_excel(writer, sheet_name='Constituents summary', index=True)
      yearly_df.to_excel(writer, sheet_name='Constituents calendar year returns', index=True)
      perf_summary.to_excel(writer, sheet_name='Perf summary', index=True)
      port_yearly_ret.to_excel(writer, sheet_name='Calendar year returns', index=True)
      df_ports_alloc_latest.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage).to_excel(writer, sheet_name='Latest alloc', index=True)
      df_ports_alloc_avg.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage).to_excel(writer, sheet_name='Average alloc', index=True)
      assetclass_breakdown_all.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage).to_excel(writer, sheet_name='Asset class exposure', index=True)
      country_breakdown_all.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage).to_excel(writer, sheet_name='Country exposure', index=True)
      sector_breakdown_all.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage).to_excel(writer, sheet_name='Secotr exposure', index=True)

      # country_diff_df.to_excel(writer, sheet_name='Country Relative', index=True)
      # sector_diff_df.to_excel(writer, sheet_name='Sector Relative', index=True)
      # df_factor_exp.to_excel(writer, sheet_name='ETF Factor MSCI', index=True)
      # df_factor_ff_exp.to_excel(writer, sheet_name='ETF Factor FF', index=True)
      # df_factor_port_exp.to_excel(writer, sheet_name='Portfolio Factor MSCI', index=True)
      # df_factor_ff_port_exp.to_excel(writer, sheet_name='Portfolio Factor FF', index=True)


      for n in port_names:
          all_port_wgt[n].to_excel(writer, sheet_name=n+'_hist_wgt', index=True)

      print(f"Excel file saved to: Projects/Projects/{file_path}")

# Create a button to trigger the save function
save_button = widgets.Button(description="Save as Excel")

# Define the button's on_click behavior
def on_save_button_clicked(b):
    save_to_excel(file_path_widget.value)

save_button.on_click(on_save_button_clicked)
display(save_button) # Display the button

Text(value='Portfolio_Visualizer_Output.xlsx', description='File Name:')

Button(description='Save as Excel', style=ButtonStyle())

Excel file saved to: Projects/Projects/Portfolio_Visualizer_Output.xlsx
